## 1.0 Setup


In [1]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier 

# set random seed to ensure that results are repeatable
np.random.seed(1)

## 2.0 Load data 

In [2]:
# load data
universal = pd.read_csv("./UniversalBank.csv")

## 3.0 Cleaning the data

In [3]:
# generate a basic summary of the data
universal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [4]:
# generate a statistical summary of the numeric value in the data
universal.describe()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


### Summary the findings from our initial evaluation of the data

* We have no categorical variables
* We have no variables that have missing values
* There doesn't seem to be a problem with the catogorical class names.

## 4.0 Process the data

* Conduct any data prepartion that should be done *BEFORE* the data split.
* Split the data.
* Conduct any data preparation that should be done *AFTER* the data split.

### 4.1  Conduct any data prepartion that should be done *BEFORE* the data split

Tasks at this stage include:
1. Drop any columns/features 
2. Decide if you with to exclude any observations (rows) due to missing na's.
2. Conduct proper encoding of categorical variables
    1. You can transform them using dummy variable encoding, one-hot-encoding, or label encoding. 

#### Drop any columns/variables we will not be using

since our target is CD account and we dont have related columns no dropping of columns

### 4.2 Split data (train/test)

In [5]:
# split the data into validation and training set
train_df, test_df = train_test_split(universal, test_size=0.7)

# to reduce repetition in later code, create variables to represent the columns
# that are our predictors and target
target = 'CD Account'
predictors = list(universal.columns)
predictors.remove(target)

### 4.3  Conduct any data prepartion that should be done *AFTER* the data split

We will look at the following:
1) imput any missing numeric values using the mean of the variable/column
2) remove differences of scale by standardizing the numerica variables

#### Impute missing values

No columns with NA's so no need to impute

In [6]:
scaler = preprocessing.StandardScaler()
cols_to_stdize = predictors               
               

train_df[cols_to_stdize] = scaler.fit_transform(train_df[cols_to_stdize]) # train_predictors is not a numpy array
test_df[cols_to_stdize] = scaler.transform(test_df[cols_to_stdize]) # validation_target is now a series object

train_X = train_df[predictors]
train_y = train_df[target] 
test_X = test_df[predictors]
test_y = test_df[target] 

In [7]:
# Common imports
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [8]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

In [9]:
log_reg_model = LogisticRegression(penalty='none', max_iter=900)
_ = log_reg_model.fit(train_X, np.ravel(train_y))

In [10]:
model_preds = log_reg_model.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978286,1.0,0.634615,0.776471


In [12]:
#random search in log
score_measure = "recall"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'penalty': ['l2'],
    'solver': ['liblinear']
    
}

logreg_rand = LogisticRegression()

rand_search_log = RandomizedSearchCV(estimator = logreg_rand, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search_log.fit(train_X, train_y)

print(f"The best {score_measure} score is {rand_search_log.best_score_}")
print(f"... with parameters: {rand_search_log.best_params_}")

bestRecallTree = rand_search_log.best_estimator_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 4 is smaller than n_iter=500. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 4 candidates, totalling 20 fits
The best recall score is 0.6801169590643275
... with parameters: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.1}


In [13]:
model_preds = rand_search_log.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_search_log_l2", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978286,1.0,0.634615,0.776471
0,rand_search_log_l2,0.977429,1.0,0.620192,0.765579


In [14]:
#grid search in log
score_measure = "recall"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'penalty': ['l1'],
    'solver': ['liblinear']
    
}

logreg_grid = LogisticRegression()
grid_search_log = GridSearchCV(logreg_grid, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search_log.fit(train_X, train_y)

print(f"The best {score_measure} score is {grid_search_log.best_score_}")
print(f"... with parameters: {grid_search_log.best_params_}")

bestRecallTree = grid_search_log.best_estimator_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
The best recall score is 0.6801169590643275
... with parameters: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}


In [15]:
model_preds = grid_search_log.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_search_log_l1", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978286,1.0,0.634615,0.776471
0,rand_search_log_l2,0.977429,1.0,0.620192,0.765579
0,grid_search_log_l1,0.978286,1.0,0.634615,0.776471


In [16]:
#svm with linear kernel
svm_lin_model = SVC(kernel="linear",probability=True)
_ = svm_lin_model.fit(train_X, np.ravel(train_y))

In [17]:
model_preds = svm_lin_model.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"lin svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978286,1.0,0.634615,0.776471
0,rand_search_log_l2,0.977429,1.0,0.620192,0.765579
0,grid_search_log_l1,0.978286,1.0,0.634615,0.776471
0,lin svm,0.978286,1.0,0.634615,0.776471


In [18]:
#svm with rbf kernel
svm_rbf_model = SVC(kernel="rbf", C=10, gamma='scale',probability=True)
_ = svm_rbf_model.fit(train_X, np.ravel(train_y))

In [19]:
model_preds = svm_rbf_model.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rbf svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978286,1.000000,0.634615,0.776471
0,rand_search_log_l2,0.977429,1.000000,0.620192,0.765579
0,grid_search_log_l1,0.978286,1.000000,0.634615,0.776471
0,lin svm,0.978286,1.000000,0.634615,0.776471
0,rbf svm,0.973714,0.876623,0.649038,0.745856


In [20]:
#svm with poly kernel
svm_poly_model = SVC(kernel="poly",probability=True, degree=3, coef0=1, C=10)
_ = svm_poly_model.fit(train_X, np.ravel(train_y))

In [21]:
model_preds = svm_poly_model.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"poly svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [22]:
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978286,1.000000,0.634615,0.776471
0,rand_search_log_l2,0.977429,1.000000,0.620192,0.765579
0,grid_search_log_l1,0.978286,1.000000,0.634615,0.776471
0,lin svm,0.978286,1.000000,0.634615,0.776471
0,rbf svm,0.973714,0.876623,0.649038,0.745856
0,poly svm,0.964857,0.748538,0.615385,0.675462


In [23]:
#random search in SVM
score_measure = "recall"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'gamma':[1,0.1,0.01,0.001],
    'kernel':['poly']
    
}

SVM_R_out = SVC()
rand_search = RandomizedSearchCV(estimator = SVM_R_out, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 16 is smaller than n_iter=500. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 16 candidates, totalling 80 fits
The best recall score is 0.6380116959064328
... with parameters: {'kernel': 'poly', 'gamma': 0.1, 'C': 10}


In [24]:
model_preds = rand_search.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_poly_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [25]:
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978286,1.000000,0.634615,0.776471
0,rand_search_log_l2,0.977429,1.000000,0.620192,0.765579
0,grid_search_log_l1,0.978286,1.000000,0.634615,0.776471
0,lin svm,0.978286,1.000000,0.634615,0.776471
0,rbf svm,0.973714,0.876623,0.649038,0.745856
0,poly svm,0.964857,0.748538,0.615385,0.675462
0,rand_poly_svm,0.960286,0.684492,0.615385,0.648101


In [27]:
#grid search in SVM
score_measure = "recall"
kfolds = 5

param_grid = {
    'C':[0.1,1,10,100],
    'gamma':[1,0.1,0.01,0.001],
    'kernel':['poly']
    
}

SVM_G_out = SVC()
grid_search = GridSearchCV(estimator = SVM_G_out, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(train_X, train_y)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 16 candidates, totalling 80 fits
The best recall score is 0.6380116959064328
... with parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'poly'}


In [28]:
model_preds = grid_search.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_poly_svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978286,1.000000,0.634615,0.776471
0,rand_search_log_l2,0.977429,1.000000,0.620192,0.765579
0,grid_search_log_l1,0.978286,1.000000,0.634615,0.776471
0,lin svm,0.978286,1.000000,0.634615,0.776471
0,rbf svm,0.973714,0.876623,0.649038,0.745856
0,poly svm,0.964857,0.748538,0.615385,0.675462
0,rand_poly_svm,0.960286,0.684492,0.615385,0.648101
0,grid_poly_svm,0.960286,0.684492,0.615385,0.648101


In [29]:
#random with tree
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,42),  
    'min_samples_leaf': np.arange(1,42),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 42), 
    'max_depth': np.arange(1,42), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search_tree = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search_tree.fit(train_X, train_y)

print(f"The best {score_measure} score is {rand_search_tree.best_score_}")
print(f"... with parameters: {rand_search_tree.best_params_}")

bestRecallTree = rand_search_tree.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.7116959064327485
... with parameters: {'min_samples_split': 14, 'min_samples_leaf': 6, 'min_impurity_decrease': 0.0006000000000000001, 'max_leaf_nodes': 36, 'max_depth': 22, 'criterion': 'entropy'}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
35 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (

In [30]:
model_preds = rand_search_tree.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"rand_poly_tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978286,1.000000,0.634615,0.776471
0,rand_search_log_l2,0.977429,1.000000,0.620192,0.765579
0,grid_search_log_l1,0.978286,1.000000,0.634615,0.776471
0,lin svm,0.978286,1.000000,0.634615,0.776471
0,rbf svm,0.973714,0.876623,0.649038,0.745856
0,poly svm,0.964857,0.748538,0.615385,0.675462
0,rand_poly_svm,0.960286,0.684492,0.615385,0.648101
0,grid_poly_svm,0.960286,0.684492,0.615385,0.648101
0,rand_poly_tree,0.976857,0.950355,0.644231,0.767908


In [32]:
#grid with tree
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(3,6),  
    'min_samples_leaf': np.arange(3,6),
    'min_impurity_decrease': np.arange(0.0009, 0.0012,0.0001),
    'max_leaf_nodes': np.arange(36,40), 
    'max_depth': np.arange(39,42), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search_tree = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search_tree.fit(train_X, train_y)

print(f"The best {score_measure} score is {grid_search_tree.best_score_}")
print(f"... with parameters: {grid_search_tree.best_params_}")

bestRecallTree = grid_search_tree.best_estimator_

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
The best recall score is 0.7339181286549707
... with parameters: {'criterion': 'entropy', 'max_depth': 39, 'max_leaf_nodes': 36, 'min_impurity_decrease': 0.0009, 'min_samples_leaf': 5, 'min_samples_split': 3}


In [33]:
model_preds = grid_search_tree.predict(test_X)
c_matrix = confusion_matrix(test_y, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"grid_poly_tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.978286,1.000000,0.634615,0.776471
0,rand_search_log_l2,0.977429,1.000000,0.620192,0.765579
0,grid_search_log_l1,0.978286,1.000000,0.634615,0.776471
0,lin svm,0.978286,1.000000,0.634615,0.776471
0,rbf svm,0.973714,0.876623,0.649038,0.745856
0,poly svm,0.964857,0.748538,0.615385,0.675462
0,rand_poly_svm,0.960286,0.684492,0.615385,0.648101
0,grid_poly_svm,0.960286,0.684492,0.615385,0.648101
0,rand_poly_tree,0.976857,0.950355,0.644231,0.767908
0,grid_poly_tree,0.972000,0.839506,0.653846,0.735135


In [34]:
performance.sort_values(by=['Recall'])

,model,Accuracy,Precision,Recall,F1
0,poly svm,0.964857,0.748538,0.615385,0.675462
0,rand_poly_svm,0.960286,0.684492,0.615385,0.648101
0,grid_poly_svm,0.960286,0.684492,0.615385,0.648101
0,rand_search_log_l2,0.977429,1.000000,0.620192,0.765579
0,default logistic,0.978286,1.000000,0.634615,0.776471
0,grid_search_log_l1,0.978286,1.000000,0.634615,0.776471
0,lin svm,0.978286,1.000000,0.634615,0.776471
0,rand_poly_tree,0.976857,0.950355,0.644231,0.767908
0,rbf svm,0.973714,0.876623,0.649038,0.745856
0,grid_poly_tree,0.972000,0.839506,0.653846,0.735135


When comparing the different models based on measuring score of recall, from the results matrix above we can say that grid search of Dtree decision model is better performing than other models based on recall. 